In [1]:
import numpy as np
from utils import standard_transform_x, standard_transform_y, get_model, train_model, create_report, calculate_stats, log_full_norm_transform_x, log_tend_norm_transform_y, create_dataloader, create_test_dataloader
# from models import Softmax_model
from utils import add_nn_arguments_jupyter
import torch.nn as nn 
import torch
import torch.optim as optim

from sklearn.metrics import mean_squared_error, r2_score

import torch
import torch.nn as nn
import torch.nn.functional as F

In [3]:
# define full path 
path_to_data = "/home/kim/data/aerosols/aerosol_emulation_data/"

x_test = np.load(path_to_data + 'X_test.npy')
y_test = np.load(path_to_data + 'y_test.npy')

x_train = np.load(path_to_data + 'X_train.npy')
y_train = np.load(path_to_data + 'y_train.npy')

x_valid = np.load(path_to_data + 'X_val.npy')
y_valid = np.load(path_to_data + 'y_val.npy')

# Select the correct 24 columns
x_test_24 = x_test[:, 8:]
x_train_24 = x_train[:, 8:] 

y_test_24 = y_test[:, :24]
y_train_24 = y_train[:, :24]

y_valid_24 = y_valid[:, :24]
x_valid_24 = x_valid[:, 8:]

# How much has it changes between x (at t = 0)  and y (at t = 1)
y_delta_train_24 = y_train_24 - x_train_24
y_delta_test_24 = y_test_24 - x_test_24
y_delta_valid_24 = y_valid_24 - x_valid_24

# Define column indices for each of the components (24 column version)
so4_indices = [0, 1, 2, 3, 4]
bc_indices = [5, 6, 7, 8]
oc_indices = [9, 10, 11, 12]
du_indices = [13, 14, 15, 16]

# Define aerosol species and their corresponding indices
species_indices = {
    'so4': so4_indices,
    'bc': bc_indices,
    'oc': oc_indices,
    'du': du_indices
}

# What are these indices?!
extra_indices = [17, 18, 19, 20, 21, 22, 23] 

np.set_printoptions(precision = 4, suppress = True, formatter = {'all': lambda x: f'{x:.4f}'})

# Changes

- naming all small

# Log normalise inputs

- log transform and mean shift, unit variance

In [ ]:
x_train_24.min()

print("Minimums:")
for i in [x_train_24, x_valid_24, x_test_24]:
    print(i.min())

eps = 1e-5 # this was needed to avoid nan's 

log_x_train_24 = torch.log(torch.tensor(x_train_24) + eps)
log_x_test_24 = torch.log(torch.tensor(x_test_24) + eps)
log_x_valid_24 = torch.log(torch.tensor(x_valid_24) + eps)

print("Minimums after transform:")
for i in [log_x_train_24, log_x_valid_24, log_x_test_24]:
    print(i.min().item())
    print(i.mean().item())
    print(i.std().item())

log_x_all = torch.concat([log_x_train_24, log_x_valid_24, log_x_test_24])
log_x_all_mean = log_x_all.mean()
log_x_all_std = log_x_all.std()

log_normal_x_train_24 = (log_x_train_24 - log_x_all_mean) / log_x_all_std
log_normal_x_valid_24 = (log_x_valid_24 - log_x_all_mean) / log_x_all_std
log_normal_x_test_24 = (log_x_test_24 - log_x_all_mean) / log_x_all_std

# Zero mean, unit variance
# for i in [log_normal_x_train_24, log_normal_x_valid_24, log_normal_x_test_24]:
#    print(i.min().item())
#    print(i.mean().item())
#    print(i.std().item())

Minimums:
-2.581566417609888e-07
-3.05456625897805e-06
-9.278067564932184e-08
Minimums after transform:
-11.53908020172387
-1.984672416393465
9.809724904629391
-11.877426129997255
-2.0807140259563837
9.714804239436738
-11.522246841896973
-1.908599175383279
9.676389281675641
-0.9791231387296909
0.0005118504712963903
1.0058132295517237
-1.0138145102136675
-0.009335512469551938
0.9960807995652592
-0.9773971763631318
0.00831181152695907
0.9921420272650052


In [ ]:
# shift between cases
log_normal_x_train_24[:, so4_indices].mean(), log_normal_x_train_24[:, so4_indices].std()

(tensor(1.5542, dtype=torch.float64), tensor(0.7384, dtype=torch.float64))

# Variance is different per index

In [ ]:
y_delta_train_24[:, so4_indices].mean(), y_delta_train_24[:, so4_indices].std()

log_y_delta_train_24_so4 = np.sign(y_delta_train_24[:, so4_indices]) * np.log1p(np.abs(y_delta_train_24[:, so4_indices]))
print(log_y_delta_train_24_so4.mean(), log_y_delta_train_24_so4.std())

log_normal_y_delta_train_24 = (log_y_delta_train_24_so4 - log_y_delta_train_24_so4.mean()) / log_y_delta_train_24_so4.std()

log_normal_y_delta_train_24_so4

0.8755111914364307 7.807200006225339


array([[-0.1127, -0.1148, -0.1219, -0.0994, -0.1121],
       [-0.1127, -0.1148, -0.1219, -0.0994, -0.1121],
       [-0.1127, -0.1148, -0.1219, -0.0994, -0.1121],
       ...,
       [-0.1121, -0.8695, -0.8440, 0.7217, -0.1094],
       [-0.1121, -0.8694, -0.8441, 0.7217, -0.1093],
       [-0.1121, -0.8699, -0.8458, 0.7227, -0.1091]])

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(
    log_normal_x_train_24[:, so4_indices].numpy(), 
    log_normal_y_delta_train_24, 
    alpha = 0.5, 
    label = 'SO4'
)

plt.xlabel('log(X_train)')
plt.ylabel('log(y_delta_train)')
plt.title(f'LOG(y_delta_train) over LOG(X_train) for {SO4}')
plt.legend()
plt.grid(True, alpha = 0.1)

plt.show()

NameError: name 'species' is not defined

Error in callback <function _draw_all_if_interactive at 0x7fec1c372440> (for post_execute), with arguments args (),kwargs {}:


KeyboardInterrupt: 